# CH02 프롬프트(Prompt)   
검색기에서 검색된 문서들을 바탕으로 언어 모델이 사용할 질문이나 명령을 생성하는 과정    
검색된 정보를 바탕으로 최종 사용자의 질문에 가장 잘 대응할 수 있는 응답을 생성하기 위해 필수적인 단계   

프롬프트의 필요성
1. 문맥(context) 설정 : 언어 모델이 특정 문맥에서 작동하도록 설정하는 역할. 모델은 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성
2. 정보 통합 : 여러 문서에서 검색된 정보는 서로 다른 관점이나 내용을 포함할 수 있음.  프롬프트 단계에서 이러한 정보를 통합하고, 모델이 이를 효율적으로 활용할 수 있는 형식으로 조정
3. 응답 품질 향상 : 질문에 대한 모델의 응답 품질은 프롬프트의 구성에 크게 의존. 잘 구성된 프롬프트는 모델이 보다 정확하고 유용한 정보를 제공하게 도움


RAG 프롬프트 구조
- 지시사항(Instruction)
- 질문(사용자 입력 질문)
- 문맥(검색된 정보)

ex)
```python 
당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요. 
만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. 
한국어로 대답하세요.

#Question: 
{이곳에 사용자가 입력한 질문이 삽입됩니다}

#Context: 
{이곳에 검색된 정보가 삽입됩니다}
```

프롬프트의 중요성
- 최적화된 방식으로 응답을 생성할 수 있으며, 시스템 전체의 성능과 사용자 만족도에 직접적인 영향을 미침


### 01. 프롬프트(Prompt)
**PromptTemplate**   
방법1. from_template()메소드를 사용하여 PromptTemplate 객체 생성   
- 치환될 변수를 {변수}로 묶어서 템플릿을 정의

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

template = "{country}의 수도는 어디인가요?"

prompt=PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [3]:
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [4]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)

# ChatGroq 모델 초기화
llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0.1,
    max_tokens=2048,
    api_key=api_key
)

chain= prompt | llm

In [5]:
chain.invoke("대한민국").content

'대한민국의 수도는 **서울**입니다. \n'

방법2. PromptTemplate 객체 생성과 동시에 prompt 생성   
- 추가 유효성 검사를 위해 input_variables 를 명시적으로 지정
- 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생

In [6]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [7]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [8]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [9]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [10]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [11]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [12]:
chain = prompt_partial | llm

In [13]:
chain.invoke("대한민국").content

'* 대한민국 수도: **서울**\n* 캐나다 수도: **오타와** \n\n\n'

In [14]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 **서울**이고, 호주의 수도는 **캔버라**입니다. \n\n\n'

partial_variables : 부분 변수 채움
- 함수를 부분적으로 사용하는 것
- 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우
- ex) 날짜, 시간

In [15]:
from datetime import datetime

datetime.now().strftime("%B %d")

'January 08'

In [16]:
def get_today():
    return datetime.now().strftime("%B %d")

In [17]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today 
    },
)

In [18]:
prompt.format(n=3)

'오늘의 날짜는 January 08 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [19]:
chain = prompt | llm

In [20]:
print(chain.invoke(3).content)

오늘(January 8일) 생일인 유명인 3명은 다음과 같습니다.

* **박보검** (1992년 1월 8일) - 배우
* **김태희** (1980년 1월 8일) - 가수, 배우
* **이병헌** (1976년 1월 8일) - 배우 





In [21]:
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

오늘(Jan 02) 생일인 유명인 3명은 다음과 같습니다.

* **박지훈** (1993-01-02) - 대한민국의 배우
* **김민석** (1993-01-02) - 대한민국의 배우
* **이정재** (1977-01-02) - 대한민국의 배우






**파일로부터 tempalte 읽어오기**   


In [23]:
from langchain_teddynote.prompts import load_prompt
prompt = load_prompt("prompts/fruit_color.yaml")


In [24]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [25]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



**ChatPromptTemplate**
- 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있음   
- 메세지는 튜플 형식으로 구성하며, (role, message)로 구성하여 리스트로 생성할 수 있음

In [26]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [27]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [28]:
from langchain_core.prompts import ChatPromptTemplate

# ChatGroq 모델 초기화
llm = ChatGroq(
    model="gemma2-9b-it",
    api_key=api_key
)

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, system
        ("system","당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human","반가워요!"),
        ("ai","안녕하세요! 무엇을 도와드릴까요?"),
        ("human","{user_input}"),
    ]
)

messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [29]:
llm.invoke(messages).content

'저는 테디라고 합니다. 😊  반가워요!\n'

In [30]:
chain = chat_template|llm

In [31]:
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Teddy입니다! 😊\n\n무엇을 도와드릴까요? 🧸'

**MessagePlaceholder**

- LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlacehloder를 제공

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt


ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

conversation 대화목록을 나중에 추가하고자 할 때 MesagePlaceholder를 사용할 수 있음

In [35]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [36]:
chain = chat_prompt | llm | StrOutputParser()

In [37]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'새로운 AI, 테디, 만남, 요약, 시작'